# Profiling MongoDB

In this notebook, we are going to profile the MongoDB with 1 million queries insertion and retreival.

In [1]:
import pymongo
from pprint import pprint

In [2]:
client = pymongo.MongoClient("mongodb://192.168.3.7:27017")

In [3]:
db = client.texttech

In [4]:
pprint(db.command("serverStatus"))

{'asserts': {'msg': 0,
             'regular': 0,
             'rollovers': 0,
             'tripwire': 0,
             'user': 610,
             'warning': 0},
 'catalogStats': {'capped': 0,
                  'collections': 1,
                  'internalCollections': 3,
                  'internalViews': 0,
                  'timeseries': 0,
                  'views': 0},
 'connections': {'active': 2,
                 'available': 838857,
                 'awaitingTopologyChanges': 1,
                 'current': 3,
                 'exhaustHello': 0,
                 'exhaustIsMaster': 0,
                 'threaded': 3,
                 'totalCreated': 143},
 'electionMetrics': {'averageCatchUpOps': 0.0,
                     'catchUpTakeover': {'called': 0, 'successful': 0},
                     'electionTimeout': {'called': 0, 'successful': 0},
                     'freezeTimeout': {'called': 0, 'successful': 0},
                     'numCatchUps': 0,
                     'numCatchUp

In [5]:
dir(db)

['_BaseObject__codec_options',
 '_BaseObject__read_concern',
 '_BaseObject__read_preference',
 '_BaseObject__write_concern',
 '_Database__client',
 '_Database__incoming_copying_manipulators',
 '_Database__incoming_manipulators',
 '_Database__name',
 '_Database__outgoing_copying_manipulators',
 '_Database__outgoing_manipulators',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply_incoming_copying_manipulators',
 '_apply_incoming_manipulators',
 '_command',
 '_create_or_update_user',
 '_current_op',
 '_default_role',
 '_fix_incoming',
 '_fix_outgoing',
 '_list_collections',
 '_read_preference_for

## Inserting values into the DB

In [6]:
import os
import xmltodict
import json

In [7]:
files = os.listdir("./data/interspeech/")
sorted(files)

['2010.xml',
 '2011.xml',
 '2012.xml',
 '2013.xml',
 '2014.xml',
 '2015.xml',
 '2016.xml',
 '2017.xml',
 '2018.xml',
 '2019.xml',
 '2020.xml',
 '2021.xml']

In [8]:
res = xmltodict.parse(open(os.path.join("./data/interspeech/", files[0])).read())

In [9]:
res["items"]["item"]

[OrderedDict([('title', 'Still talking to machines (cognitively speaking)'),
              ('author', 'Steve Young'),
              ('abstract',
               'This overview article reviews the structure of a fully statistical spoken dialogue system (SDS), using as illustration, various systems and components built at Cambridge over the last few years. Most of the components in an SDS are essentially classifiers which can be trained using supervised learning. However, the dialogue management component must track the state of the dialogue and optimise a reward accumulated over time. This requires techniques for statistical inference and policy optimisation using reinforcement learning. The potential advantages of a fully statistical SDS are the ability to train from data without hand-crafting, increased robustness to environmental noise and user uncertainty, and the ability to adapt and learn on-line.'),
              ('url',
               'https://www.isca-speech.org/archive/pdfs/int

In [12]:
total_docs=0
result = []
for file in files:
    res = xmltodict.parse(open(os.path.join("./data/interspeech/", file)).read())
    total_docs+=len(res["items"]["item"])
    result.extend(res["items"]["item"])
    print(file, len(res["items"]["item"]))
result = [dict(i) for i in result]
print(total_docs)

2010.xml 781
2011.xml 848
2012.xml 682
2013.xml 792
2014.xml 640
2015.xml 792
2016.xml 817
2017.xml 844
2018.xml 791
2019.xml 970
2020.xml 1035
2021.xml 997
9989


In [11]:
titles = [i["title"] for i in result]
authors = [i["author"] for i in result]
abstracts = [i["abstract"] for i in result]
urls = [i["url"] for i in result]

In [12]:
from random import randint
from tqdm.autonotebook import tqdm

<ipython-input-12-9f03e60e58d1>:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [13]:
%%time

for x in tqdm(range(1, 1000002), total=1000000):
    paper = {
        'title' : titles[randint(0, (len(titles)-1))] + ' ' + titles[randint(0, (len(titles)-1))],
        'author' : authors[randint(0, (len(authors)-1))] + ' ' + authors[randint(0, (len(authors)-1))],
        'abstract' : abstracts[randint(0, (len(abstracts)-1))]
    }

    r=db.TextTechnology.insert_one(paper)

print('finished creating one million papers indices')

  0%|          | 0/1000000 [00:00<?, ?it/s]

KeyboardInterrupt: 